In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam

def create_model(
    input_shape,
    cnn_filters=[64, 128],
    cnn_kernel_sizes=[3, 3],
    cnn_pool_sizes=[2, 2],
    lstm_units=[50, 50],
    dense_units=[50],
    dropout_rate=0.5,
    learning_rate=0.001
):
    inputs = Input(shape=input_shape)

    x = inputs
    # Добавляем CNN слои
    for filters, kernel_size, pool_size in zip(cnn_filters, cnn_kernel_sizes, cnn_pool_sizes):
        x = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(x)
        x = MaxPooling1D(pool_size=pool_size)(x)

    x = TimeDistributed(Flatten())(x)

    # Добавляем LSTM слои
    for units in lstm_units:
        x = LSTM(units, return_sequences=True)(x)

    # Добавляем полносвязные слои
    for units in dense_units:
        x = Dense(units, activation='relu')(x)
        x = Dropout(dropout_rate)(x)

    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    return model

# Пример использования
input_shape = (100, 1)  # Зависит от ваших данных
model = create_model(
    input_shape=input_shape,
    cnn_filters=[32, 64],
    cnn_kernel_sizes=[5, 3],
    cnn_pool_sizes=[2, 2],
    lstm_units=[50, 100],
    dense_units=[50, 20],
    dropout_rate=0.3,
    learning_rate=0.0001
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 96, 32)         │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 48, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 46, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 23, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 23, 64)         │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 23, 50)         │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 23, 100)        │        60,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 23, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23, 20)         │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 23, 20)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 23, 1)          │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,891 (374.57 KB)

 Trainable params: 95,891 (374.57 KB)

 Non-trainable params: 0 (0.00 B)